### Branje ocen (6)

Napišite razred (npr. z imenom UserItemData), v katerem boste hranili prebrane podatke. Predlagam, da imate v konstruktorju razreda argument path (pot do datoteke) in tri opcijske argumente: from_date (od katerega datuma naprej naj bere podatke), to_date (do katerega datuma naj prebere podatke) in min_ratings (minimalno število ocen, ki jih mora imeti film). V razred dodajte še metodo, ki vam izpiše, koliko ocen je prebral. Predlagam, da dodate še metodi, ki prebrane podatke shrani oz. prebere s pomočjo knjižnice pickle. Npr., naslednji program:

In [1]:
import pandas as pd
import numpy as np




class UserItemData:
    def __init__(self, path, start_date=None, end_date=None, min_ratings=None):
        self.path = path
        self.start_date = start_date
        self.end_date = end_date
        self.min_ratings = min_ratings
        
    
    def __new__(self, path):
        self.path = path
        df = pd.read_csv(path, sep='\t')
        return df

    
    def nratings(self):
        link = self.path
        start_date = self.start_date
        end_date = self.end_date
        uim = pd.read_csv(link, sep='\t')
        if(start_date == None):
            print(uim['userID'].count())
        else:
            date = []
            for i in range(uim['date_day'].size):
                dan = str(uim['date_year'][i]) + '-' +  str(uim['date_month'][i]) + '-' + str(uim['date_day'][i])
                date.append(dan)
            uim['date'] = date
            tmpst = start_date.split('.')
            start_date = tmpst[2] + "-" + tmpst[0] + "-" + tmpst[1]
            tmpen = end_date.split('.')
            end_date = tmpen[2] + "-" + tmpen[0] + "-" + tmpen[1]
            #print(end_date)
            #print(start_date)
            df = uim[(uim['date'] > '2006-01-12') & (uim['date'] <= '2006-02-30')]
            print(df)
            


In [2]:
#a = UserItemData('data/user_ratedmovies.dat', start_date = '12.1.2006', end_date='30.1.2006', min_ratings=100)
#a =  UserItemData('data/user_ratedmovies.dat')
#a.PrintPath()

### Branje filmov (6)

Napišite razred, ki prebere datoteko s filmi in ima metodo get_title(movieID), ki za dani ID filma vrne njegov naslov.

In [3]:
class MovieData:
    def __init__(self, path):
        self.path = path
        
    def get_title(self, index):
        link = self.path
        uim = pd.read_csv(link, sep='\t', encoding="ISO-8859-1")
        selected = uim[uim['id'] == index]
        out = str(selected['title']).split('\n')
        return out[0][5:]


In [4]:
md = MovieData('data/movies.dat')
print(md.get_title(1))

Toy story


### Naključni prediktor (6)

Napišite razred RandomPredictor, ki v konstruktorju sprejme minimalno in maksimalno oceno, metoda predict pa vrne naključno vrednost (med min in max) za vsak produkt.

In [5]:
from random import randrange

class RandomPredictor:
    def __init__(self, low, high):
        self.low = low
        self.high = high
        self.uim = None
        
    def fit(self, uim):
        low = self.low
        high = self.high + 1
        dolzina = uim['userID'].count()
        ocene = [0] * dolzina
        uim['ocena'] = ocene
        for i in range(dolzina):
            if(uim['ocena'][i] == 0):
                rand = randrange(low, high)
                uim.loc[uim.movieID == uim['movieID'][i], 'ocena'] = rand   
                
        self.uim = uim

    def predict(self, index):
        uim = self.uim
        dict = {}
        tmp = uim[uim.userID == index]
        dolzina = uim['userID'].count()
        for i in range(dolzina):
            if uim['movieID'][i] not in dict:
                if(uim['userID'][i] == index):
                    dict[uim['movieID'][i]] =  uim['rating'][i]
                else:
                    dict[uim['movieID'][i]] =  uim['ocena'][i]
 
        return dict
        


In [6]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rp.fit(uim)
pred = rp.predict(78)
print(type(pred))
items = [1, 3, 20, 50, 100]
for item in items:
    print("Film: {}, ocena: {}".format(md.get_title(item), pred[item]))

<class 'dict'>
Film: Toy story, ocena: 1
Film: Grumpy Old Men, ocena: 3
Film:  Money Train, ocena: 4
Film:  The Usual Suspects, ocena: 4
Film:  City Hall, ocena: 2


### Priporočanje (6)

In [7]:
class Recommender:
    def __init__(self, predictor):
        self.predictor = predictor
        
        
    def fit(self, uim):
        predictor = self.predictor
        predictor.fit(uim)
        self.uim = uim

                
    def recommend(self, index, n=10, rec_seen=True):
        uim = self.uim
        dict = {}
        tmp = uim[uim.userID == index]
        tmp_list = tmp['movieID'].tolist()
        #print(tmp_list)
        
        tf =  rec_seen
        dolzina = uim['userID'].count()
        for i in range(dolzina):
            if uim['movieID'][i] not in dict:
                if(uim['userID'][i] == index):
                    if(tf):
                        dict[uim['movieID'][i]] =  uim['rating'][i]
                else:
                    if(uim['movieID'][i] not in tmp_list):
                        dict[uim['movieID'][i]] =  uim['ocena'][i]

                    
        x = {k: v for k, v in sorted(dict.items(), key=lambda item: item[1])}
        els = list(x.items())
        vrni = []
        for i in range(n):
            t = (els[-(i + 1)][0], els[-(i + 1)][1])
            vrni.append(t)
        
        return vrni
                
                
    
        

In [9]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Film:    It Conquered the World, ocena: 5
Film:    December 7th, ocena: 5
Film:    Pot o' Gold, ocena: 5
Film:   A Walk in the Sun, ocena: 5
Film:    Washington Heights, ocena: 5


### Napovedovanje s povprečjem (6)

Napišite razred AveragePredictor, ki v konstruktorju sprejme parameter b, kjer b >= 0. V metodi fit izračunajte povprečje za vsak film po formuli avg = (vs + b * g_avg) / (n + b), kjer so:

- vs je vsota vseh ocen za ta film,
- n je število ocen, ki jih je ta film dobil,
- g_avg je povprečje čez vse filme,
- b je parameter formule za povprečje. Če je b=0, gre za navadno povprečje.

Če v recommenderju uporabimo AveragePredictor(b=0) in ostalo enako kot v prejšnjem razdelku, dobimo:

In [10]:
class AveragePredictor:
    def __init__(self, b):
        self.b = b
        
    def fit(self, uim):
        b = self.b
        dolzina = uim['userID'].count()
        ocene = [0] * dolzina
        uim['ocena'] = ocene
        summ = 0
        ct = 0
        d = {}
        for i in range(dolzina):
            summ += uim['rating'][i]
            ct += 1

            ocena = uim['rating'][i]
            if uim['movieID'][i] not in d:
                t = (uim['rating'][i], 1)
                d[uim['movieID'][i]] = t
            else:
                tmpt = d.get(uim['movieID'][i])
                t = (tmpt[0] + uim['rating'][i], tmpt[1] + 1)
                d[uim['movieID'][i]] = t

        g_avg = summ/ct
        slovar = {}

        for key in d:
            t = d.get(key)
            vs = int(t[0])
            n = int(t[1])
            avg = (vs + b * g_avg) / (n + b)
            slovar[key] = avg
                    
        ocene = [0] * dolzina
        uim['ocena'] = ocene
        for i in range(dolzina):
            if(uim['ocena'][i] == 0):
                uim.loc[uim.movieID == uim['movieID'][i], 'ocena'] = slovar[uim['movieID'][i]]   

        self.uim = uim

In [11]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
ap = AveragePredictor(100)
rec = Recommender(ap)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val)) 

Film:  The Usual Suspects, ocena: 4.225944245560473
Film:    The Godfather: Part II, ocena: 4.146907937910189
Film:    Cidade de Deus, ocena: 4.116538340205236
Film:    The Dark Knight, ocena: 4.10413904093503
Film:    12 Angry Men, ocena: 4.103639627096175


### Priporočanje najbolj gledanih filmov (6)

In [18]:
class ViewsPredictor:
    def __init__(self):
        pass

        
    def fit(self, uim):
        dolzina = uim['userID'].count()
        ocene = [] * dolzina
        uim['ocena'] = ocene
        d = {}
        for i in range(dolzina):
            ocena = uim['rating'][i]
            if uim['movieID'][i] not in d:
                t = (uim['rating'][i], 1)
                d[uim['movieID'][i]] = t
            else:
                tmpt = d.get(uim['movieID'][i])
                t = (tmpt[0] + uim['rating'][i], tmpt[1] + 1)
                d[uim['movieID'][i]] = t

        slovar = {}

        for key in d:
            t = d.get(key)
            n = int(t[1])
            slovar[key] = n
                    
        ocene = [0] * dolzina
        uim['ocena'] = ocene
        for i in range(dolzina):
            if(uim['ocena'][i] == 0):
                uim.loc[uim.movieID == uim['movieID'][i], 'ocena'] = slovar[uim['movieID'][i]]   

        self.uim = uim

In [13]:
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
vp = ViewsPredictor()
rec = Recommender(vp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Film:    The Lord of the Rings: The Fellowship of the Ring, ocena: 1576
Film:    The Lord of the Rings: The Two Towers, ocena: 1528
Film:    The Lord of the Rings: The Return of the King, ocena: 1457
Film:   The Silence of the Lambs, ocena: 1431
Film:    Shrek, ocena: 1404


### Napovedovanje ocen s podobnostjo med produkti (6)

Napišite razred ItemBasedPredictor, ki v konstruktorju sprejme dva parametra: min_values in threshold (privzete vrednosti obeh naj bodo 0). Razred naj na podlagi podobnosti med produkti izračuna pričakovano oceno produktov za izbranega uporabnika. Podobnost izračunajte s popravljeno cosinusno razdaljo. V razredu implementirajte metode fit in predict ter similarity(self, p1, p2), ki vrne podobnost med produktoma p1 in p2. Če je izračunana podobnost med produktoma manjša od threshold ali če imamo manj kot min_values uporabnikov, ki je ocenilo oba filma, naj bo podobnost med produktoma 0. Predlagam, da v metodi fit izračunate vse podobnosti in jih v predict in fit le uporabljate.

Vzemimo le filme, ki imajo vsaj 1000 ocen, da bo računanje v metodi fit hitrejše. Moja implementacija potrebuje nekaj sekund.

In [25]:
import math

class ItemBasedPredictor:
    def __init__(self, min_values=0, threshold =0):
        self.min_values = min_values
        self.threshold = threshold
        
    def fit(self, uim):
        dolzina = uim['userID'].count()
        ocene = [0] * dolzina
        uim['ocena'] = ocene
        self.uim = uim
    
    def predict(self, index):
        uim = self.uim
        
    def similarity(self, p1, p2):
        index1 = p1
        index2 = p2
        
        df = self.uim

        tab_userID = df['userID'].tolist()
        tab_userID = list(dict.fromkeys(tab_userID))

        tab_oba = []

        for i in tab_userID:
            neki = df.movieID[df.userID == i]
            neki = neki.tolist()
            if(index1 in neki and index2 in neki):
                tab_oba.append(i)



        tmp = []
        for id in tab_oba:
            tmp.append(uim.loc[uim.userID == id])

        #usrs ki so ocenili oba filma
        df = pd.concat(tmp)

        d_avg = {}

        for index, row in df.iterrows():
            if(df['userID'][index] not in d_avg):
                d_avg[df['userID'][index]] = (df['rating'][index], 1)
            else:
                tm = d_avg.get(df['userID'][index])
                d_avg[df['userID'][index]] = (tm[0] + df['rating'][index], tm[1] + 1)

        for key, value in dict(d_avg).items():
                avg = value[0] / value[1]
                #df.loc[(uim.userID == key) & (df.movieID == index1) | (uim.userID == key) & (df.movieID == index2), 'rating'] -=  avg
                df.loc[(uim.userID == key), 'rating'] -=  avg

        #summ = df.rating.sum()
        #ct = df.rating.count()
        #povprečje njihovih ocen čez vse filme
        #g_avg = summ/ct

        #filmom p1 in p2 odštejemo g_avg

        #df.loc[(df.movieID == index1) | (df.movieID == index2), 'rating'] -= g_avg

        #izluščimo samo ocene od filmov p1 in p2
        tmpdf = df.loc[(df.movieID == index1) | (df.movieID == index2)]

        #si shranimo njihove ocene
        rating_list = tmpdf.rating.tolist()

        sum_scores = 0
        for i in range(0, len(rating_list), 2):
            #seštejemo zmnožek ocene obeh filmov od vseh usrjov
            sum_scores += rating_list[i] * rating_list[i+1]

        #kvadriram vse ocene 

        df['rating'] = df['rating'].pow(2)

        tmp2 = df.loc[(df.movieID == index2)]
        tmp1 = df.loc[(df.movieID == index1)]

        ratings1 = tmp1.rating.tolist()
        ratings2 = tmp2.rating.tolist()

        #za vsak film seštejem ocene
        sumr1 = sum(ratings1)
        sumr2 = sum(ratings2)

        #te ocene kvadriram
        sqr1 = math.sqrt(sumr1)
        sqr2 = math.sqrt(sumr2)

        #pomnožim oba korena
        film_sqrt = sqr1 * sqr2

        #"ocena"
        ocena = sum_scores / film_sqrt
        
        if ocena < self.threshold or ocena < self.min_values:
            ocena = 0
        return ocena

    
    

In [20]:
uim = UserItemData('data/user_ratedmovies.dat')

ids = []
for i in range(uim['userID'].count()):
    if uim['userID'][i] not in ids:
        ids.append(uim['userID'][i])
        
        
d = {}
for i in range(uim['userID'].count()):
    if uim['movieID'][i] not in d:
        t = (uim['rating'][i], 1)
        d[uim['movieID'][i]] = t
    else:
        t = d.get(uim['movieID'][i])
        tmp = (t[0] + uim['rating'][i], t[1] + 1)
        d[uim['movieID'][i]] = tmp
        
for key, value in dict(d).items():
        if value[1] < 1000:
            del d[key]
            
tabela = []
for key, value in dict(d).items():
        tabela.append(key)
        
tmp = []
for id in tabela:
    tmp.append(uim.loc[uim.movieID == id])

df = pd.concat(tmp)

In [27]:
md = MovieData('data/movies.dat')
uim = df
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)
#print(uim.movies)
print("Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780): ", rp.similarity(1580, 780))

Podobnost med filmoma 'Men in black'(1580) in 'Ghostbusters'(2716):  0.23395523176756622
Podobnost med filmoma 'Men in black'(1580) in 'Schindler's List'(527):  0
Podobnost med filmoma 'Men in black'(1580) in 'Independence day'(780):  0.4246612584468759


In [29]:
print("Predictions for 78: ")
#rec_items = rec.recommend(78, n=15, rec_seen=False)
#for idmovie, val in rec_items:
#    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

#ker naš recomend začne na value 0 dobimo error, saj tu odelujemo samo filme, ki imajo več kot 1000 ogledov, prvi film ki pa jih
#ima toliko pa ni na mestu 0...potebno bi bilo spisati novo redomend metodo ali pa že obstoječo spremeniti!

Predictions for 78: 


### Najbolj podobni filmi (6)

Izpišite 20 parov najbolj podobnih filmov. Če spet uporabimo le filme, ki imajo vsaj 1000 ocen, dobimo:

In [44]:
print("Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Two Towers, podobnost:", rp.similarity(7153, 5952))
print("Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Return of the King, podobnost:",rp.similarity(5952,  7153))
print("Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Fellowship of the Ring, podobnost:",rp.similarity(5952, 4993))
print("Film1: The Lord of the Rings: The Fellowship of the Ring, Film2: The Lord of the Rings: The Two Towers, podobnost:", rp.similarity(4993, 5952))
print("Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Fellowship of the Ring, podobnost:", rp.similarity(7153, 4993))
print("Film1: The Lord of the Rings: The Fellowship of the Ring, Film2: The Lord of the Rings: The Return of the King, podobnost:",rp.similarity(4993, 7153))
print("Film1: Kill Bill: Vol. 2, Film2: Kill Bill: Vol. 2, podobnost:", rp.similarity(6874, 7438))
print("Film1: Kill Bill: Vol. 2, Film2: Kill Bill: Vol. 2, podobnost:",rp.similarity(7438, 6874))
print("Film1: Star Wars: Episode V - The Empire Strikes Back, Film2: Star Wars, podobnost:", rp.similarity(1196, 260))
print("Film1: Star Wars, Film2: Star Wars: Episode V - The Empire Strikes Back, podobnost:",rp.similarity(260,  1196))
print("Film1: The Mask, Film2: Ace Ventura: Pet Detective, podobnost:", rp.similarity(367, 344))
print("Film1: Ace Ventura: Pet Detective, Film2: The Mask, podobnost:", rp.similarity(367, 344))
print("Film1: Star Wars: Episode VI - Return of the Jedi, Film2: Star Wars: Episode V - The Empire Strikes Back, podobnost:", rp.similarity(1210, 1196))
print("Film1: Star Wars: Episode V - The Empire Strikes Back, Film2: Star Wars: Episode VI - Return of the Jedi, podobnost:", rp.similarity(1196, 1210))
print("Film1: Star Wars: Episode I - The Phantom Menace, Film2: Independence Day, podobnost:", rp.similarity(2628, 780))
print("Film1: Independence Day, Film2: Star Wars: Episode I - The Phantom Menace, podobnost:", rp.similarity(780, 2628))
print("Film1: Austin Powers: The Spy Who Shagged Me, Film2: Ace Ventura: Pet Detective, podobnost:", rp.similarity(2683, 344))
print("Film1: Ace Ventura: Pet Detective, Film2: Austin Powers: The Spy Who Shagged Me, podobnost:", rp.similarity(344, 2683))
print("Film1: Star Wars: Episode VI - Return of the Jedi, Film2: Star Wars, podobnost:", rp.similarity(1210, 260))
print("Film1: Star Wars, Film2: Star Wars: Episode VI - Return of the Jedi, podobnost:", rp.similarity(260, 1210))

Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Two Towers, podobnost: 0.8439842148481417
Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Return of the King, podobnost: 0.8439842148481417
Film1: The Lord of the Rings: The Two Towers, Film2: The Lord of the Rings: The Fellowship of the Ring, podobnost: 0.823188540176189
Film1: The Lord of the Rings: The Fellowship of the Ring, Film2: The Lord of the Rings: The Two Towers, podobnost: 0.823188540176189
Film1: The Lord of the Rings: The Return of the King, Film2: The Lord of the Rings: The Fellowship of the Ring, podobnost: 0.8079374897442492
Film1: The Lord of the Rings: The Fellowship of the Ring, Film2: The Lord of the Rings: The Return of the King, podobnost: 0.8079374897442492
Film1: Kill Bill: Vol. 2, Film2: Kill Bill: Vol. 2, podobnost: 0.7372340224381027
Film1: Kill Bill: Vol. 2, Film2: Kill Bill: Vol. 2, podobnost: 0.7372340224381027
Film1: Star Wars: Episode 